# Treating missing values on metadata

## 1) Introduction: 
It uses the movie metadata from Kaggle from the link below. The meta data has missing values and they were filled with median value for the group after removing the duplicate rows.
<br>https://www.kaggle.com/datasets/karrrimba/movie-metadatacsv

In [1]:
# Set up path for data files
from google.colab import drive
drive.mount("/content/drive" )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/Pract/'

In [3]:
# Libraries are imported
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
from statsmodels.stats import weightstats as stests

### 1.1) Reading and exploring the data set movie_metadata.csv, which contains data about films from IMDb (Internet Movie Database)

In [4]:
# Read the data
df = pd.read_csv(path + 'movie_metadata.csv' )

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   duration                   5028 non-null   float64
 4   director_facebook_likes    4939 non-null   float64
 5   actor_3_facebook_likes     5020 non-null   float64
 6   actor_2_name               5030 non-null   object 
 7   actor_1_facebook_likes     5036 non-null   float64
 8   gross                      4159 non-null   float64
 9   genres                     5043 non-null   object 
 10  actor_1_name               5036 non-null   object 
 11  movie_title                5043 non-null   object 
 12  num_voted_users            5043 non-null   int64  
 13  cast_total_facebook_likes  5043 non-null   int64

In [6]:
df.shape

(5043, 28)

In [7]:
df.head(3)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0$,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0$,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0$,2015.0,393.0,6.8,2.35,85000


In [8]:
# Checking for columns with null values
df.isna().sum()

color                         19
director_name                104
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               303
budget                         0
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
dtype: int64

In [9]:
# Counting for number of unique values
df.nunique()

color                           2
director_name                2398
num_critic_for_reviews        528
duration                      191
director_facebook_likes       435
actor_3_facebook_likes        906
actor_2_name                 3032
actor_1_facebook_likes        878
gross                        4035
genres                        914
actor_1_name                 2097
movie_title                  4917
num_voted_users              4826
cast_total_facebook_likes    3978
actor_3_name                 3521
facenumber_in_poster           19
plot_keywords                4760
movie_imdb_link              4919
num_user_for_reviews          954
language                       47
country                        65
content_rating                 18
budget                        440
title_year                     91
actor_2_facebook_likes        917
imdb_score                     78
aspect_ratio                   22
movie_facebook_likes          876
dtype: int64

In [10]:
# Checking for unique conuntires
df.country.unique()

array(['USA', 'UK', nan, 'New Zealand', 'Canada', 'Australia', 'Belgium',
       'Japan', 'Germany', 'China', 'France', 'New Line', 'Mexico',
       'Spain', 'Hong Kong', 'Czech Republic', 'India', 'Soviet Union',
       'South Korea', 'Peru', 'Italy', 'Russia', 'Aruba', 'Denmark',
       'Libya', 'Ireland', 'South Africa', 'Iceland', 'Switzerland',
       'Romania', 'West Germany', 'Chile', 'Netherlands', 'Hungary',
       'Panama', 'Greece', 'Sweden', 'Norway', 'Taiwan', 'Official site',
       'Cambodia', 'Thailand', 'Slovakia', 'Bulgaria', 'Iran', 'Poland',
       'Georgia', 'Turkey', 'Nigeria', 'Brazil', 'Finland', 'Bahamas',
       'Argentina', 'Colombia', 'Israel', 'Egypt', 'Kyrgyzstan',
       'Indonesia', 'Pakistan', 'Slovenia', 'Afghanistan',
       'Dominican Republic', 'Cameroon', 'United Arab Emirates', 'Kenya',
       'Philippines'], dtype=object)

In [11]:
# Checking for duplicated rows
print(f'Total number of duplicated rows are: {len(df[df.duplicated()])}' )

Total number of duplicated rows are: 45


In [12]:
# Examining the duplicated rows
df[df.duplicated()].head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
137,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,Action|Adventure|Drama|Romance,...,239.0,English,USA,PG-13,180000000.0$,2016.0,10000.0,6.6,2.35,29000
187,Color,Bill Condon,322.0,115.0,386.0,12000.0,Kristen Stewart,21000.0,292298923.0,Adventure|Drama|Fantasy|Romance,...,329.0,English,USA,PG-13,120000000.0$,2012.0,17000.0,5.5,2.35,65000
204,Color,Hideaki Anno,1.0,120.0,28.0,12.0,Shin'ya Tsukamoto,544.0,NaN,Action|Adventure|Drama|Horror|Sci-Fi,...,13.0,Japanese,Japan,NaN,0.0$,2016.0,106.0,8.2,2.35,0
303,Color,Joe Wright,256.0,111.0,456.0,394.0,Cara Delevingne,20000.0,34964818.0,Adventure|Family|Fantasy,...,186.0,English,USA,PG,150000000.0$,2015.0,548.0,5.8,2.35,24000
389,Color,Josh Trank,369.0,100.0,128.0,78.0,Reg E. Cathey,596.0,56114221.0,Action|Adventure|Sci-Fi,...,695.0,English,USA,PG-13,120000000.0$,2015.0,360.0,4.3,2.35,41000


In [13]:
# Dropping all the duplicates
df.drop_duplicates(inplace=True)

In [14]:
# Checking for duplicates
len(df[df.duplicated()])

0

In [15]:
df.shape

(4998, 28)

After removing the 45 duplicate rows, there are a total of 4,998 rows.

### 1.2) Determining the missing values in duration column 

In [16]:
df.duration.isna().sum()


15

There are 15 missing values in duration column.

### 1.3) Replacing the missing values in the `duration` column with the median for a particular movie genre (Ex: missing value in duration for movie that belongs to Comedy genre are replaced with median value of duration of all movies in Comedy)

In [17]:
# Examning the median values for each Genres
df.groupby(['genres'])['duration'].median()

genres
Action                                                             107.0
Action|Adventure                                                   116.5
Action|Adventure|Animation|Comedy|Crime|Family|Fantasy              95.0
Action|Adventure|Animation|Comedy|Drama|Family|Fantasy|Thriller     45.0
Action|Adventure|Animation|Comedy|Drama|Family|Sci-Fi               96.0
                                                                   ...  
Sci-Fi|Thriller                                                    108.0
Thriller                                                            99.0
Thriller|War                                                       133.0
Thriller|Western                                                    85.0
Western                                                            104.0
Name: duration, Length: 914, dtype: float64

In [18]:
# Checking a single median value
df.groupby(['genres'])['duration'].median()['Action']

107.0

In [19]:
# Checking a single median value
df.groupby(['genres'])['duration'].median()['Documentary']

91.5

In [20]:
# Find all the rows needs to be filled with the median value
df[df['duration'].isna() ]

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,0.0$,NaN,12.0,7.1,NaN,0
199,Color,Matt Birch,1.0,NaN,0.0,159.0,Dave Legeno,10000.0,NaN,Action|Fantasy,...,2.0,English,UK,NaN,0.0$,2011.0,570.0,7.5,NaN,40
206,Color,Matt Birch,4.0,NaN,0.0,1000.0,Toby Jones,10000.0,NaN,Fantasy,...,2.0,English,UK,NaN,0.0$,2010.0,2000.0,6.4,NaN,25
1510,Color,Tony Kaye,NaN,NaN,194.0,816.0,Aisha Tyler,970.0,NaN,Crime|Drama,...,NaN,English,USA,NaN,23000000.0$,2009.0,856.0,7.2,NaN,26
3604,Color,NaN,10.0,NaN,NaN,502.0,Tuppence Middleton,1000.0,NaN,Drama|History|Romance|War,...,44.0,English,UK,TV-14,0.0$,NaN,888.0,8.2,16.00,11000
3815,Color,Marc Bennett,2.0,NaN,3.0,723.0,Natasha Henstridge,975.0,NaN,Comedy|Drama,...,2.0,English,USA,NaN,5000000.0$,2012.0,900.0,6.7,NaN,35
3834,Color,Shekar,1.0,NaN,0.0,0.0,Diganth,5.0,NaN,Comedy|Romance,...,1.0,Kannada,India,NaN,0.0$,2013.0,0.0,7.1,NaN,2
4299,Color,Kundan Shah,1.0,NaN,4.0,72.0,Bobby Deol,353.0,NaN,Action|Comedy|Romance|Thriller,...,1.0,Hindi,India,NaN,0.0$,2000.0,89.0,2.8,NaN,10
4392,Color,David M. Matthews,5.0,NaN,8.0,394.0,Lamman Rucker,713.0,2592808.0,Crime|Drama|Thriller,...,15.0,English,USA,R,0.0$,2010.0,607.0,3.5,2.35,588
4397,Color,Romesh Sharma,4.0,NaN,0.0,45.0,Revathy,421.0,129319.0,Romance,...,4.0,English,India,NaN,70000000.0$,2005.0,96.0,5.1,NaN,9


In [21]:
df.groupby('genres')['duration']

In [22]:
# Replacing with the median 
df['duration'] = df.groupby('genres')['duration'].transform(
    lambda grp: grp.fillna(grp.median())
)

In [23]:
# Checking if `duration` value for the genres of 'Documentary' is 91.5
df.loc[4]

color                                                                      NaN
director_name                                                      Doug Walker
num_critic_for_reviews                                                     NaN
duration                                                                  91.5
director_facebook_likes                                                  131.0
actor_3_facebook_likes                                                     NaN
actor_2_name                                                        Rob Walker
actor_1_facebook_likes                                                   131.0
gross                                                                      NaN
genres                                                             Documentary
actor_1_name                                                       Doug Walker
movie_title                  Star Wars: Episode VII - The Force Awakens    ...
num_voted_users                                     

In [24]:
# Checking for any missing values for duration
df[df['duration'].isna() ]

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes


## 1.4) As can be seen above that there are no missing values for 'duration' columns. Also examining 'Documentary' generes, correct median value was applied. 